In [2]:
import torch
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(variable_name: str, derivative_tensor: torch.Tensor, tensor: torch.Tensor) -> bool:
  ex = torch.all(derivative_tensor == tensor.grad).item()
  app = torch.allclose(derivative_tensor, tensor.grad)
  maxdiff = (derivative_tensor - tensor.grad).abs().max().item()
  print(f'{variable_name:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')
  return app

In [3]:
sample_size = 10
input_neurons = 1
hidden_neurons = 2
output_features = 1
iterations = 100
learning_rate = .1
target_number = 2
g = torch.Generator().manual_seed(2147483647) # for reproducibility

X = torch.arange(-5, sample_size * input_neurons -5, dtype=torch.float32, requires_grad=True).view(sample_size, -1)
W1 = torch.ones(input_neurons, hidden_neurons)
W2 = torch.ones(hidden_neurons, output_features)
b1 = torch.zeros(hidden_neurons, dtype=torch.float32)
b2 = torch.zeros(output_features, dtype=torch.float32)
parameters = [W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True
for i in range(iterations):
    # forward pass
    # first layer
    h1 = X @ W1
    hb1 = h1 + b1
    a1 = hb1.relu()
    
    # second layer
    h2 = a1 @ W2
    hb2 = h2 + b2
    a2 = hb2.relu()
    losses = (target_number - a2).abs()
    total_loss = losses.mean()
    
    # backward pass
    for p in parameters:
        p.grad = None
    for t in [total_loss, losses, h1, hb1, a1, h2, hb2, a2,  W2, b2, X]:
        t.retain_grad()
    total_loss.backward()
    d_total_loss = torch.tensor(1)
    d_losses = torch.ones_like(losses) * d_total_loss/sample_size
    d_a2 = -torch.sign(target_number-a2) * d_losses
    d_hb2 = d_a2.clone()
    d_hb2[hb2<=0] = 0
    d_b2 = d_hb2.sum(0)
    d_h2 = d_hb2
    d_W2 = a1.T @ d_h2 
    d_a1 = d_h2 @ W2.T
    d_hb1 = d_a1.clone()
    d_hb1[a1<=0] = 0
    d_b1 = d_hb1.sum(0)
    d_h1 = d_hb1
    d_W1 = X.T @ d_h1
    d_X = d_h1 @ W1.T
    
    manual_grads = [d_W1, d_b1, d_W2, d_b2]
    
    d_hb = d_losses
    matches = [
        cmp('losses', d_losses, losses),
        cmp('a2', d_a2, a2),
        cmp('hb2', d_hb2, hb2),
        cmp('b2', d_b2, b2),
        cmp('h2', d_h2, h2),
        cmp('d_W2', d_W2, W2),
        cmp('d_a1', d_a1, a1),
        cmp('d_hb1', d_hb1, hb1),
        cmp('d_b1', d_b1, b1),
        cmp('d_h1', d_h1, h1),
        cmp('d_W1', d_W1, W1),
        # cmp('d_X', d_X, X)
    ]
    
    
    print(f'{i+1}/{iterations}: {total_loss=}')
    
    # updates
    for p, m in zip(parameters, manual_grads):
        auto_update = -learning_rate * p.grad
        manual_update = -learning_rate * m
        assert(False not in matches)
        p.data += manual_update


losses          | exact: True  | approximate: True  | maxdiff: 0.0
a2              | exact: True  | approximate: True  | maxdiff: 0.0
hb2             | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
h2              | exact: True  | approximate: True  | maxdiff: 0.0
d_W2            | exact: True  | approximate: True  | maxdiff: 0.0
d_a1            | exact: True  | approximate: True  | maxdiff: 0.0
d_hb1           | exact: True  | approximate: True  | maxdiff: 0.0
d_b1            | exact: True  | approximate: True  | maxdiff: 0.0
d_h1            | exact: True  | approximate: True  | maxdiff: 0.0
d_W1            | exact: True  | approximate: True  | maxdiff: 0.0
1/100: total_loss=tensor(2.4000, grad_fn=<MeanBackward0>)
losses          | exact: True  | approximate: True  | maxdiff: 0.0
a2              | exact: True  | approximate: True  | maxdiff: 0.0
hb2             | exact: True  | approximate: True  | maxdiff: 0.0
b2  

In [4]:
xg = torch.tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.1656],
        [ 0.3656],
        [ 0.3656],
        [ 0.3656]])

xg_bad = torch.tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.1656],
        [ 0.1656],
        [ 0.1656],
        [ 0.1656]])

wg = torch.tensor([[0.7280, 0.7280]])

hg = torch.tensor([[ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0910, -0.0910],
        [ 0.0910,  0.0910],
        [ 0.0910,  0.0910],
        [ 0.0910,  0.0910]])


wg @ hg.T

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.1325,  0.1325,
          0.1325,  0.1325]])

In [5]:
a = torch.arange(1,10).view(3,-1)
a[0][1]

tensor(2)

In [24]:
k = torch.randn(100000000)*5

In [27]:
k.std()

tensor(5.0003)

In [37]:
a = torch.ones((3,2)) * 3
a

tensor([[3., 3.],
        [3., 3.],
        [3., 3.]])

In [41]:
b = torch.ones(8).view(2,2,2) * 2
b


tensor([[[2., 2.],
         [2., 2.]],

        [[2., 2.],
         [2., 2.]]])

In [42]:
a @ b

tensor([[[12., 12.],
         [12., 12.],
         [12., 12.]],

        [[12., 12.],
         [12., 12.],
         [12., 12.]]])

In [17]:
ones = torch.arange(9, dtype=torch.float32).view(3,-1)
print(ones)
masked = torch.tril(ones)
print(torch.sum(masked, 1, keepdim=True))
print(masked.sum(1, keepdim=True))

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
tensor([[ 0.],
        [ 7.],
        [21.]])
tensor([[ 0.],
        [ 7.],
        [21.]])


In [18]:
masked

tensor([[0., 0., 0.],
        [3., 4., 0.],
        [6., 7., 8.]])

In [25]:
import torch.nn.functional as F
import torch.nn as nn

F.softmax(masked)

/var/folders/1r/b2w4br1j7hvbs91lnp4ycbxw0000gn/T/ipykernel_9606/244397608.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(masked)


tensor([[0.3333, 0.3333, 0.3333],
        [0.2654, 0.7214, 0.0132],
        [0.0900, 0.2447, 0.6652]])

In [41]:

torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
affinity_weights = q @ k.transpose(-2, -1) * head_size**-.5# (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril = torch.tril(torch.ones(T,T))
affinity_weights = affinity_weights.masked_fill(tril == 0, float('-inf'))
affinity_weights = F.softmax(affinity_weights, dim=2)

v = value(x) # (B, T, head_size)
out = affinity_weights @ v

out.shape

torch.Size([4, 8, 16])

In [42]:
affinity_weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
         [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
         [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
         [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4016, 0.5984, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3365, 0.2271, 0.4364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3019, 0.2060, 0.2899, 0.2022, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1058, 0.1700, 0.1530, 0.3451, 0.2261, 0.0000, 0.0000, 0.0000],
         [0.1526, 0.164